# test with Receipe of Kongbao chicken
- https://www.onceuponachef.com/recipes/kung-pao-chicken.html

In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
dotenv.dotenv_values()

In [ ]:
!pip install -U requests

In [ ]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(request_timeout=60)

In [ ]:

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../files/receipe_for_kongbao.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
docs

In [ ]:
# # If nothing stored before
es_store = ElasticsearchStore.from_documents(
    docs, 
    embeddings, 
    es_url="http://localhost:9200", 
    index_name="myindex",
    distance_strategy="COSINE"
    # distance_strategy="EUCLIDEAN_DISTANCE"
    # distance_strategy="DOT_PRODUCT"
)

In [ ]:
## for loading form previos database
es_store = ElasticsearchStore(
    index_name="myindex",
    embedding=embeddings, 
    es_url="http://localhost:9200", 
    distance_strategy="COSINE"
    # distance_strategy="EUCLIDEAN_DISTANCE"
    # distance_strategy="DOT_PRODUCT"
)

In [ ]:
query = "How would the chicken be heated?"
docs = es_store.similarity_search(query)
docs

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="map_reduce")

chain = ConversationalRetrievalChain(
    retriever=es_store.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

In [ ]:
chat_history = []
query = "What are the vegetables needed?"
result = chain({"question": query, "chat_history": chat_history})

In [ ]:
result['answer']

In [ ]:
chat_history

# Ref
- https://python.langchain.com/docs/integrations/vectorstores/elasticsearch
- https://python.langchain.com/docs/use_cases/qa_structured/integrations/elasticsearch